In [0]:
sc

<SparkContext master=spark://10.139.64.5:7077 appName=Databricks Shell>

In [0]:
spark

### create a dataframe

In [0]:
green_TaxiDF = spark.read.csv("/Volumes/azuredatabrickss/default/greentaxi/2021_Green_Taxi_Trip_Data.csv",header=True,inferSchema=True)

In [0]:
repartitioned_df=green_TaxiDF.repartition(10)

In [0]:
repartitioned_df.write.mode("append").parquet("/Volumes/azuredatabrickss/default/greentaxi/")

In [0]:
print(green_TaxiDF.schema)

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge', Doubl

In [0]:
green_TaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [0]:
# create REadStream
taxiReadStream =spark.readStream.format("parquet").schema(green_TaxiDF.schema).option("header","true").load("/Volumes/azuredatabrickss/default/readstream/")
# readstream transformation,structured streaming transformation
# taxiTransformation =taxiReadStream.groupBy("trip_type").sum("fare_amount")



In [0]:
taxiReadStream=spark.readStream.format("parquet").schema(green_TaxiDF.schema).option("header","true").load("/Volumes/azuredatabrickss/default/readstream")
#2.convert string to timestamp
from pyspark.sql.functions import col,to_timestamp,window
taxiReadStream=taxiReadStream.withColumn("lpep_pickup_datetime",to_timestamp(col("lpep_pickup_datetime"),"yyyy-MM-dd HH:mm:ss"))
#3.add watermark and perform aggregation (window of aggregate oprn)
taxitransformation=taxiReadStream.withWatermark("lpep_pickup_datetime","10 minutes")\
    .groupBy(window(col("lpep_pickup_datetime"),"10 minutes"),col("trip_type")).sum("fare_amount")

#4.write 
taxitransformation.writeStream.outputMode("append")\
    .format("parquet")\
        .option("path","/Volumes/azuredatabrickss/default/writestream")\
        .option("checkpointLocation","/Volumes/azuredatabrickss/default/checkpoint")\
        .start().awaitTermination() 

In [0]:
taxiReadStream.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [0]:
taxiReadStream=spark.readStream.format("parquet").schema(green_TaxiDF.schema).option("header","true").load("/Volumes/azuredatabrickss/default/readstream")
#2.convert string to timestamp
from pyspark.sql.functions import col,to_timestamp,window
taxiReadStream=taxiReadStream.withColumn("lpep_pickup_datetime",to_timestamp(col("lpep_pickup_datetime"),"yyyy-MM-dd HH:mm:ss"))
#3.add watermark with 10 minutes 
# and perform aggregation (window of aggregate oprn)
taxitransformation=taxiReadStream.withWatermark("lpep_pickup_datetime","10 minutes")

taxitransformation.createOrReplaceTempView("taxitable")
#5)add watermark  and perform the aggregation 
# query = spark.sql("select * from taxitable where trip_type=1")
query = spark.sql("""
                  select lpep_pickup_datetime,trip_type,count(*) from taxitable
                  group by lpep_pickup_datetime,trip_type""")

#4.write 
query.writeStream.outputMode("append")\
    .format("parquet")\
        .option("path","/Volumes/azuredatabrickss/default/writestream1/")\
        .option("checkpointLocation","/Volumes/azuredatabrickss/default/checkpoint1")\
        .start().awaitTermination() 


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:436)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1266)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:983)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:669)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:687)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.Usag

In [0]:
taxiReadStream = spark.readStream.format("parquet") \
    .schema(green_TaxiDF.schema) \
    .option("header", "true") \
    .load("/Volumes/azuredatabricks/default/readstream/")
from pyspark.sql.functions import col,to_timestamp,window
taxiReadStream=taxiReadStream.withColumn("lpep_pickup_datetime",to_timestamp(col("lpep_pickup_datetime"),"yyyy-MM-dd HH:mm:ss"))
taxitransformation=taxiReadStream.withWatermark("lpep_pickup_datetime","10 minutes")

taxitransformation.createOrReplaceTempView("taxi")

query = spark.sql("SELECT lpep_pickup_datetime,trip_type,count(*)  FROM taxi group by lpep_pickup_datetime,trip_type")

query.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azuredatabrickss/default/writestream1/") \
    .option("checkpointLocation", "/Volumes/azuredatabrickss/default/checkpoint1") \
    .start() \
    .awaitTermination()